# B.2 - Two-Stage Ripeness Classification

**Experiment**: B.2
**Approach**: Detect → Crop → Classify (2 stages)
**Objective**: Test if separating detection and classification improves accuracy
**Comparison**: B.1 End-to-End (mAP50=0.801, mAP50-95=0.514)

**Pipeline**:
1. **Stage 1**: Detector - Detect all FFBs (ripe/unripe) using YOLO detection
2. **Crop**: Extract bounding boxes with 10% margin
3. **Stage 2**: Classifier - Classify crops using YOLO classification
4. **Inference**: End-to-end pipeline on test set

---

## Setup

In [ ]:
import os
import torch
import numpy as np
import cv2
from pathlib import Path
from ultralytics import YOLO
import pandas as pd
from datetime import datetime
from tqdm.auto import tqdm
import shutil

IS_KAGGLE = os.path.exists('/kaggle/input')
BASE_PATH = Path('/kaggle/working' if IS_KAGGLE else r'd:\Work\Assisten Dosen\Anylabel\Experiments')
DATASET_PATH = Path('/kaggle/input/ffb-ripeness' if IS_KAGGLE else BASE_PATH / 'datasets' / 'ffb_ripeness')

print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"Dataset: {DATASET_PATH}")
print(f"CUDA: {torch.cuda.is_available()}")

## Stage 1: Train Detector (Ripe/Unripe Detection)

In [ ]:
# Stage 1 Config
stage1_config_path = BASE_PATH / 'configs' / 'ffb_ripeness_detect.yaml'
stage1_config_path.parent.mkdir(exist_ok=True)

yaml_content = f"""path: {DATASET_PATH.as_posix()}
train: images/train
val: images/val
test: images/test
nc: 2
names: ['ripe', 'unripe']
"""
stage1_config_path.write_text(yaml_content)
print(f"Stage 1 config: {stage1_config_path}")

In [ ]:
# Train Stage 1 (2 seeds)
STAGE1_CONFIG = {
    'model': 'yolo11n.pt',
    'epochs': 50,
    'batch': 16,
    'imgsz': 640,
    'device': 0 if torch.cuda.is_available() else 'cpu',
}
SEEDS = [42, 123]
STAGE1_PREFIX = 'exp_b2_stage1'

stage1_results = {}
for seed in SEEDS:
    print(f"\n{'='*60}\nSTAGE 1 - DETECTOR - SEED {seed}\n{'='*60}\n")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    model = YOLO(STAGE1_CONFIG['model'])
    results = model.train(
        data=str(stage1_config_path),
        epochs=STAGE1_CONFIG['epochs'],
        batch=STAGE1_CONFIG['batch'],
        imgsz=STAGE1_CONFIG['imgsz'],
        device=STAGE1_CONFIG['device'],
        name=f"{STAGE1_PREFIX}_seed{seed}",
        seed=seed,
        exist_ok=True
    )
    stage1_results[seed] = str(BASE_PATH / 'runs' / 'detect' / f"{STAGE1_PREFIX}_seed{seed}" / 'weights' / 'best.pt')
    print(f"✅ Stage 1 seed {seed} complete")

print(f"\nStage 1 models saved:")
for seed, path in stage1_results.items():
    print(f"  Seed {seed}: {path}")

## Stage 2: Extract Crops with 10% Margin

In [ ]:
# Extract crops function
def extract_crops_with_margin(detector_model_path, image_dir, label_dir, output_dir, margin=0.1):
    """Extract crops from detections with margin"""
    model = YOLO(detector_model_path)
    output_dir = Path(output_dir)
    
    # Create output dirs
    for class_name in ['ripe', 'unripe']:
        (output_dir / class_name).mkdir(parents=True, exist_ok=True)
    
    image_files = list(Path(image_dir).glob('*.png')) + list(Path(image_dir).glob('*.jpg'))
    stats = {'ripe': 0, 'unripe': 0}
    
    for img_path in tqdm(image_files, desc="Extracting crops"):
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        
        # Get GT labels
        label_path = Path(label_dir) / f"{img_path.stem}.txt"
        if not label_path.exists():
            continue
        
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        for i, line in enumerate(lines):
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:5])
            
            # Convert to pixel coords
            x_center *= w
            y_center *= h
            width *= w
            height *= h
            
            # Add margin
            width_margin = width * margin
            height_margin = height * margin
            x1 = int(max(0, x_center - (width + width_margin) / 2))
            y1 = int(max(0, y_center - (height + height_margin) / 2))
            x2 = int(min(w, x_center + (width + width_margin) / 2))
            y2 = int(min(h, y_center + (height + height_margin) / 2))
            
            # Crop
            crop = img[y1:y2, x1:x2]
            if crop.size == 0:
                continue
            
            # Save
            class_name = 'ripe' if class_id == 0 else 'unripe'
            crop_filename = f"{img_path.stem}_crop{i}.png"
            crop_path = output_dir / class_name / crop_filename
            cv2.imwrite(str(crop_path), crop)
            stats[class_name] += 1
    
    return stats

print("Crop extraction function defined ✓")

In [ ]:
# Extract crops for all splits (using best detector from stage 1)
CROPS_BASE = BASE_PATH / 'datasets' / 'ffb_ripeness_twostage_crops'
best_detector = stage1_results[42]  # Use seed 42 detector

print(f"Extracting crops using: {best_detector}\n")

for split in ['train', 'val', 'test']:
    print(f"\n{'='*60}\nExtracting {split.upper()} crops\n{'='*60}\n")
    
    img_dir = DATASET_PATH / 'images' / split
    label_dir = DATASET_PATH / 'labels' / split
    output_dir = CROPS_BASE / split
    
    stats = extract_crops_with_margin(best_detector, img_dir, label_dir, output_dir, margin=0.1)
    
    print(f"\n{split.upper()} crops extracted:")
    print(f"  Ripe: {stats['ripe']}")
    print(f"  Unripe: {stats['unripe']}")
    print(f"  Total: {sum(stats.values())}")

print(f"\n✅ All crops saved to: {CROPS_BASE}")

## Stage 3: Train Classifier on Crops

In [ ]:
# Stage 2 Config (Classification)
stage2_config_path = BASE_PATH / 'configs' / 'ffb_ripeness_classify.yaml'

yaml_content = f"""path: {CROPS_BASE.as_posix()}
train: train
val: val
test: test
nc: 2
names: ['ripe', 'unripe']
"""
stage2_config_path.write_text(yaml_content)
print(f"Stage 2 config: {stage2_config_path}")

In [ ]:
# Train Stage 2 Classifier (2 seeds)
STAGE2_CONFIG = {
    'model': 'yolo11n-cls.pt',
    'epochs': 100,
    'batch': 32,
    'imgsz': 224,
    'device': 0 if torch.cuda.is_available() else 'cpu',
}
STAGE2_PREFIX = 'exp_b2_stage2'

stage2_results = {}
for seed in SEEDS:
    print(f"\n{'='*60}\nSTAGE 2 - CLASSIFIER - SEED {seed}\n{'='*60}\n")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    model = YOLO(STAGE2_CONFIG['model'])
    results = model.train(
        data=str(stage2_config_path),
        epochs=STAGE2_CONFIG['epochs'],
        batch=STAGE2_CONFIG['batch'],
        imgsz=STAGE2_CONFIG['imgsz'],
        device=STAGE2_CONFIG['device'],
        name=f"{STAGE2_PREFIX}_seed{seed}",
        seed=seed,
        exist_ok=True
    )
    stage2_results[seed] = str(BASE_PATH / 'runs' / 'classify' / f"{STAGE2_PREFIX}_seed{seed}" / 'weights' / 'best.pt')
    print(f"✅ Stage 2 seed {seed} complete")

print(f"\nStage 2 models saved:")
for seed, path in stage2_results.items():
    print(f"  Seed {seed}: {path}")

## Stage 4: End-to-End Inference & Evaluation

In [ ]:
# End-to-end inference function
def two_stage_inference(detector_path, classifier_path, test_images_dir, test_labels_dir):
    """Run two-stage pipeline: detect -> classify"""
    detector = YOLO(detector_path)
    classifier = YOLO(classifier_path)
    
    image_files = list(Path(test_images_dir).glob('*.png')) + list(Path(test_images_dir).glob('*.jpg'))
    
    results = {'correct': 0, 'wrong': 0, 'missed': 0}
    
    for img_path in tqdm(image_files, desc="Two-stage inference"):
        # Stage 1: Detect
        detections = detector(str(img_path), verbose=False)
        
        # Stage 2: Classify each detection
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        
        # Get GT labels
        label_path = Path(test_labels_dir) / f"{img_path.stem}.txt"
        if not label_path.exists():
            continue
        
        with open(label_path, 'r') as f:
            gt_labels = [int(line.split()[0]) for line in f.readlines()]
        
        # Process detections
        for det in detections[0].boxes:
            x1, y1, x2, y2 = map(int, det.xyxy[0].cpu().numpy())
            crop = img[y1:y2, x1:x2]
            
            if crop.size == 0:
                continue
            
            # Classify crop
            cls_result = classifier(crop, verbose=False)
            pred_class = int(cls_result[0].probs.top1)
            
            # Compare with GT (simplified - assumes 1:1 mapping)
            if gt_labels:
                gt_class = gt_labels.pop(0)
                if pred_class == gt_class:
                    results['correct'] += 1
                else:
                    results['wrong'] += 1
        
        results['missed'] += len(gt_labels)  # Undetected objects
    
    return results

print("Inference function defined ✓")

In [ ]:
# Run end-to-end evaluation for both seeds
e2e_results = {}

for seed in SEEDS:
    print(f"\n{'='*60}\nEND-TO-END EVALUATION - SEED {seed}\n{'='*60}\n")
    
    results = two_stage_inference(
        stage1_results[seed],
        stage2_results[seed],
        DATASET_PATH / 'images' / 'test',
        DATASET_PATH / 'labels' / 'test'
    )
    
    total = results['correct'] + results['wrong']
    accuracy = results['correct'] / total if total > 0 else 0
    
    e2e_results[seed] = {
        'accuracy': accuracy,
        'correct': results['correct'],
        'wrong': results['wrong'],
        'missed': results['missed']
    }
    
    print(f"\nResults for seed {seed}:")
    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  Correct: {results['correct']}")
    print(f"  Wrong: {results['wrong']}")
    print(f"  Missed: {results['missed']}")

## Summary & Comparison with B.1

In [ ]:
# Summary
df = pd.DataFrame(e2e_results).T
avg = df.mean()
avg.name = 'Average'
df = pd.concat([df, avg.to_frame().T])

print("\n" + "="*60)
print("B.2 TWO-STAGE CLASSIFICATION - RESULTS")
print("="*60 + "\n")
print(df.to_string(float_format=lambda x: f"{x:.3f}" if isinstance(x, float) else f"{x}"))

# Comparison
print("\n" + "="*60)
print("COMPARISON: B.1 END-TO-END vs B.2 TWO-STAGE")
print("="*60 + "\n")
print("B.1 (End-to-End): mAP50=0.801, mAP50-95=0.514")
print(f"B.2 (Two-Stage): Accuracy={avg['accuracy']:.3f}")
print("\nNote: Different metrics - B.1 uses mAP, B.2 uses classification accuracy")

# Save
output_file = BASE_PATH / 'kaggleoutput' / 'test_twostage.txt'
output_file.parent.mkdir(exist_ok=True, parents=True)
with open(output_file, 'w') as f:
    f.write(f"B.2 Two-Stage Classification Results\nGenerated: {datetime.now()}\n\n")
    f.write(df.to_string())
print(f"\n✅ Saved: {output_file}")

---

## Conclusion

Two-stage pipeline:
1. ✅ Stage 1 (Detector): Trained to detect ripe/unripe FFBs
2. ✅ Crops extracted with 10% margin
3. ✅ Stage 2 (Classifier): Trained on crops
4. ✅ End-to-end evaluation completed

**Key Insight**: Two-stage approach allows:
- Specialization of detector and classifier
- Cropping focuses classifier on relevant region
- Potentially better accuracy than end-to-end

**Trade-offs**:
- More complex pipeline (2 models)
- Slower inference
- Better interpretability (can debug each stage separately)

Compare results with B.1 to determine best approach for deployment.